In [ ]:
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import pickle
import os
import json
import requests

# 0. Train a simple Linear Regression model

In [ ]:
house = datasets.load_boston()
train_x, test_x, train_y, test_y = train_test_split(house.data,
                                                    house.target,
                                                    test_size=0.2,
                                                    random_state=42)
lr = LinearRegression()
lr.fit(train_x, train_y)

# 1. Deployment on WMLA

## 1.1 Basic setup

**Make sure to set deployment name. If left unchanged, will overwrite files.**

In [ ]:
# Set deployment name
deploy_name = 'toy-model'

In [ ]:
# REST_SERVER required by dlim CLI tool
os.environ['HOST'] = 'https://wmla-console-cpd-wmla.apps.cpd.mskcc.org'
os.environ['REST_SERVER'] = os.environ['HOST'] + '/dlim/v1/'
os.environ['INFERENCE_URL'] = 'https://wmla-inference-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/inference'
    
# Add to path
if not f"{os.environ['HOME']}/bin" in os.environ['HOME']:
    os.environ['PATH'] = os.environ['PATH'] + f':{os.environ["HOME"]}/bin'

## 1.2 Create working directory

In [ ]:
os.environ['DEPLOY_NAME'] = deploy_name
os.environ['DIR_DEPLOY_SUBMISSION'] = f'/userfs/deploy_submissions/{os.environ["DEPLOY_NAME"]}'

!rm -rf $DIR_DEPLOY_SUBMISSION
!mkdir -p $DIR_DEPLOY_SUBMISSION

## 1.3 Save model files to working directory

In [ ]:
with open(f'{os.environ["DIR_DEPLOY_SUBMISSION"]}/toy-model.pkl', 'wb') as f:
    pickle.dump(lr, f)

## 1.4 Save model.json file to working directory

In [ ]:
model_profile = {
    "name" : os.environ['DEPLOY_NAME'],
    "tag" : "test",
    "weight_path" : "./",
    "model_path" : "./",
    "runtime" : "python",
    "kernel_path" : "kernel.py",
    "schema_version" : "1"
}

# Save to working directory
with open(f'{os.environ["DIR_DEPLOY_SUBMISSION"]}/model.json','w') as f:
    json.dump(model_profile, f)

## 1.5 Create README.md file

In [ ]:
with open(f'{os.environ["DIR_DEPLOY_SUBMISSION"]}/README.md','w') as f:
    f.write('Test.')

## 1.6 Save kernel.py file to working directory

In [ ]:
kernel_file = """#!/usr/bin/env python

import redhareapiversion
from redhareapi import Kernel
import json
import time
import os
from io import BytesIO
import pickle

class MatchKernel(Kernel):
    def on_kernel_start(self, kernel_context):
        try:
            Kernel.log_info("kernel input: " + kernel_context.get_model_description())
            model_desc = json.loads(kernel_context.get_model_description())
            model_file = os.path.join(model_desc['model_path'], 'toy-model.pkl')
            with open(model_file, 'rb') as f:
                self.model = pickle.load(f)
        except Exception as e:
            Kernel.log_error(str(e))
    
    def on_task_invoke(self, task_context):
        try:
            in_invoke_time = time.time()
            Kernel.log_info("on_task_invoke")
            while task_context != None:
                input_data = json.loads(task_context.get_input_data())
                req_id = input_data['id']
                #d = input_data['inputs'][0]
                data = input_data['data']
                outs = self.model.predict(data)
                output_data = {'name':'output0',
                               'data':outs.tolist()}
                task_context.set_output_data(json.dumps({"id":req_id,
                                                         "outputs":[output_data]}))
                task_context = task_context.next()
            done_invoke_time = time.time()
            Kernel.log_info("Inference cost time: %.4f" % (done_invoke_time - in_invoke_time))
        except Exception as e:
            Kernel.log_error("Do task invoke failed: {}".format(e))
    
    def on_kernel_shutdown(self):
        Kernel.log_info('on_kernel_shutdown')
        
if __name__ == '__main__':
    obj_kernel = MatchKernel()
    obj_kernel.run()"""

print(kernel_file, file=open(f'{os.environ["DIR_DEPLOY_SUBMISSION"]}/kernel.py', 'w'))

## 1.7 Submit deployment

In [ ]:
# Remove hidden files
!rm -rf $DIR_DEPLOY_SUBMISSION/.*

In [ ]:
!dlim model deploy -p $DIR_DEPLOY_SUBMISSION --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

### 1.7.1 Start deployment

In [ ]:
!dlim model start $DEPLOY_NAME --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

In [ ]:
# View status
!dlim model view $DEPLOY_NAME -a --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

## 2. Test scoring

In [ ]:
headers = {'Authorization': f'Bearer {os.getenv("USER_ACCESS_TOKEN")}'}
data = {'id':'test_data','data':test_x.tolist()}
url = f'{os.environ["INFERENCE_URL"]}/{os.environ["DEPLOY_NAME"]}'

In [ ]:
r = requests.post(url, headers=headers,
                  json = data, verify = False)
r.status_code

In [ ]:
json.dumps(data)